# Tuning RandomForestRegressor

In [2]:
import numpy
import csv
import time
import pandas
import signal
import warnings
warnings.filterwarnings("ignore")

In [3]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline

In [4]:
from sklearn.multioutput import ClassifierChain
from sklearn.multioutput import MultiOutputRegressor
from sklearn.multioutput import MultiOutputClassifier

from sklearn.ensemble.weight_boosting import AdaBoostClassifier
from sklearn.ensemble.weight_boosting import AdaBoostRegressor
from sklearn.ensemble.bagging import BaggingClassifier
from sklearn.ensemble.bagging import BaggingRegressor
from sklearn.ensemble.forest import ExtraTreesClassifier
from sklearn.ensemble.forest import ExtraTreesRegressor
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.ensemble.gradient_boosting import GradientBoostingRegressor
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.ensemble.forest import RandomForestRegressor

from sklearn.linear_model.bayes import ARDRegression
from sklearn.linear_model.bayes import BayesianRidge
from sklearn.naive_bayes import BernoulliNB
from sklearn.cross_decomposition.cca_ import CCA
from sklearn.tree.tree import DecisionTreeClassifier
from sklearn.tree.tree import DecisionTreeRegressor
from sklearn.linear_model.coordinate_descent import ElasticNet
from sklearn.tree.tree import ExtraTreeClassifier
from sklearn.tree.tree import ExtraTreeRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process.gpc import GaussianProcessClassifier
from sklearn.gaussian_process.gpr import GaussianProcessRegressor
from sklearn.linear_model.huber import HuberRegressor
from sklearn.neighbors.classification import KNeighborsClassifier
from sklearn.neighbors.regression import KNeighborsRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.semi_supervised.label_propagation import LabelPropagation
from sklearn.semi_supervised.label_propagation import LabelSpreading
from sklearn.linear_model.least_angle import Lars
from sklearn.linear_model.coordinate_descent import Lasso
from sklearn.linear_model.least_angle import LassoLars
from sklearn.linear_model.least_angle import LassoLarsIC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model.base import LinearRegression
from sklearn.svm.classes import LinearSVC
from sklearn.svm.classes import LinearSVR
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.neural_network.multilayer_perceptron import MLPClassifier
from sklearn.neural_network.multilayer_perceptron import MLPRegressor
from sklearn.linear_model.coordinate_descent import MultiTaskElasticNet
from sklearn.linear_model.coordinate_descent import MultiTaskLasso
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.svm.classes import NuSVC
from sklearn.svm.classes import NuSVR
from sklearn.linear_model.omp import OrthogonalMatchingPursuit
from sklearn.cross_decomposition.pls_ import PLSCanonical
from sklearn.cross_decomposition.pls_ import PLSRegression
from sklearn.linear_model.passive_aggressive import PassiveAggressiveClassifier
from sklearn.linear_model.passive_aggressive import PassiveAggressiveRegressor
from sklearn.linear_model.perceptron import Perceptron
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model.ransac import RANSACRegressor
from sklearn.neighbors.classification import RadiusNeighborsClassifier
from sklearn.neighbors.regression import RadiusNeighborsRegressor
from sklearn.linear_model.ridge import Ridge
from sklearn.linear_model.ridge import RidgeClassifier
from sklearn.linear_model.stochastic_gradient import SGDClassifier
from sklearn.linear_model.stochastic_gradient import SGDRegressor
from sklearn.svm.classes import SVC
from sklearn.svm.classes import SVR
from sklearn.linear_model.theil_sen import TheilSenRegressor

In [5]:
mlts = [MultiOutputRegressor, ClassifierChain]
bss = [Ridge, LinearSVR, GaussianProcessRegressor, PassiveAggressiveRegressor, KNeighborsRegressor, MLPRegressor, HuberRegressor, SGDRegressor, LassoLarsIC, ExtraTreeRegressor, ElasticNet, DecisionTreeRegressor, Perceptron, OrthogonalMatchingPursuit, BernoulliNB]

In [6]:
tfidf = pandas.read_csv('07 NM PT TFIDF.csv', delimiter=',', encoding='latin-1').fillna('')
tfidf[:3]

,____________________,_____________________,_____________________ follow,able,acceptable,accessory,accessory require,accidentally,accommodate,accompany,...,work,would,write,writer,wrong,wrong patient,wrong tattoo,xrt,xrt date,xrt pt
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
ohe = pandas.read_csv('07 NM PT OHE.csv', delimiter=',', encoding='latin-1').fillna('')
ohe[:3]

,Wrong target or OAR contours,Treatment plan acceptable but not physically deliverable,Excess imaging dose,Failure to perform on-treatment imaging as per instructions,Wrong patient,Fall or other patient injury or medical condition,Wrong anatomical site (excluding laterality),Treatment plan (isodose distribution) unacceptable,Systematic hardware/software (including dose-volume) error,Interventional procedure error (Retired value),...,Inappropriate or poorly informed decision to treat or plan,Inadequate coordination of combined modality care,"Wrong, missing, mislabeled, or damaged treatment accessories",Wrong side (laterality),Wrong prescription dose-fractionation or calculation error,Radiation therapy scheduling error,Treatment not delivered - personnel/hardware/software failure,"Wrong patient position, setup point, or shift",Wrong planning margins,Allergic reaction
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [8]:
def lowest_correct(trues, preds):
    num_of_options = len(trues)
    drop_down_options = list(reversed(numpy.argsort(preds)))
    correct_options = [i for i in range(num_of_options) if trues[i]==1]
    return max([drop_down_options.index(correct_option) for correct_option in correct_options]) + 1
def average_lowest_correct(list_of_trues, list_of_preds):
    length = len(list_of_trues)
    return numpy.mean([lowest_correct(list(list_of_trues.iloc[i]), list(list_of_preds[i])) for i in range(length)])

## Default

In [9]:
def deadline(timeout, *args):
    def decorate(f):
        def handler(signum, frame):
            raise Exception

        def new_f(*args):
            signal.signal(signal.SIGALRM, handler)
            signal.alarm(timeout)
            return f(*args)
            signal.alarm(0)

        new_f.__name__ = f.__name__
        return new_f
    return decorate

In [10]:
@deadline(500)
def cv_ensemble(model, xs, ys):
    temp = cross_validate(model, xs, ys, return_train_score=True, scoring=make_scorer(average_lowest_correct), n_jobs=1, cv=2)
    return numpy.mean(temp['test_score']).round(2)

In [10]:
cv_ensemble(MultiOutputRegressor(RandomForestRegressor()), tfidf, ohe)

4.86

## Grid Search

In [11]:
def prn_rsl(gsc):
    prms = gsc.cv_results_['params']
    tsts = (gsc.cv_results_['split0_test_score']+gsc.cv_results_['split1_test_score'])/2
    for i in numpy.argsort(tsts):
        print(prms[i], tsts[i])

In [12]:
ppl = Pipeline([('mb', MultiOutputRegressor(RandomForestRegressor()))])

In [15]:
grd = {
#     'mb__estimator__criterion': ['mae', 'mse'],
#     'mb__estimator__max_features': ['auto', 'sqrt', 'log2', None],
    'mb__estimator__bootstrap': [True, False],
    'mb__estimator__oob_score': [True, False],
}
gsc = GridSearchCV(estimator=ppl, param_grid=grd, scoring=make_scorer(average_lowest_correct), n_jobs=1, cv=2, verbose=2, error_score=0)
gsc.fit(tfidf, ohe)
prn_rsl(gsc)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] mb__estimator__bootstrap=True, mb__estimator__oob_score=True ....
[CV]  mb__estimator__bootstrap=True, mb__estimator__oob_score=True, total=  11.2s
[CV] mb__estimator__bootstrap=True, mb__estimator__oob_score=True ....


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.4s remaining:    0.0s


[CV]  mb__estimator__bootstrap=True, mb__estimator__oob_score=True, total=  10.6s
[CV] mb__estimator__bootstrap=True, mb__estimator__oob_score=False ...
[CV]  mb__estimator__bootstrap=True, mb__estimator__oob_score=False, total=   9.9s
[CV] mb__estimator__bootstrap=True, mb__estimator__oob_score=False ...
[CV]  mb__estimator__bootstrap=True, mb__estimator__oob_score=False, total=   8.5s
[CV] mb__estimator__bootstrap=False, mb__estimator__oob_score=True ...
[CV]  mb__estimator__bootstrap=False, mb__estimator__oob_score=True, total=   0.0s
[CV] mb__estimator__bootstrap=False, mb__estimator__oob_score=True ...
[CV]  mb__estimator__bootstrap=False, mb__estimator__oob_score=True, total=   0.0s
[CV] mb__estimator__bootstrap=False, mb__estimator__oob_score=False ..
[CV]  mb__estimator__bootstrap=False, mb__estimator__oob_score=False, total=  18.1s
[CV] mb__estimator__bootstrap=False, mb__estimator__oob_score=False ..
[CV]  mb__estimator__bootstrap=False, mb__estimator__oob_score=False, total=

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  1.3min finished


{'mb__estimator__bootstrap': False, 'mb__estimator__oob_score': True} 0.0
{'mb__estimator__bootstrap': True, 'mb__estimator__oob_score': True} 4.867179215270413
{'mb__estimator__bootstrap': True, 'mb__estimator__oob_score': False} 4.964803098657368
{'mb__estimator__bootstrap': False, 'mb__estimator__oob_score': False} 7.259773172529073


In [15]:
grd = {
    'mb__estimator__max_features': ['auto', 'sqrt', 'log2', None],
    'mb__estimator__bootstrap': [True],
    'mb__estimator__oob_score': [True],
}
gsc = GridSearchCV(estimator=ppl, param_grid=grd, scoring=make_scorer(average_lowest_correct), n_jobs=-1, cv=2, verbose=2, error_score=0)
gsc.fit(tfidf, ohe)
prn_rsl(gsc)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=auto, mb__estimator__oob_score=True 
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=auto, mb__estimator__oob_score=True 
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=sqrt, mb__estimator__oob_score=True 
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=sqrt, mb__estimator__oob_score=True 
[CV]  mb__estimator__bootstrap=True, mb__estimator__max_features=sqrt, mb__estimator__oob_score=True, total=   5.5s
[CV]  mb__estimator__bootstrap=True, mb__estimator__max_features=sqrt, mb__estimator__oob_score=True, total=   5.3s
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__oob_score=True 
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__oob_score=True 
[CV]  mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__oob_score=True, t

[Parallel(n_jobs=-1)]: Done   6 out of   8 | elapsed:   28.3s remaining:    9.4s


[CV]  mb__estimator__bootstrap=True, mb__estimator__max_features=None, mb__estimator__oob_score=True, total=  22.5s
[CV]  mb__estimator__bootstrap=True, mb__estimator__max_features=None, mb__estimator__oob_score=True, total=  22.5s


[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:   35.3s finished


{'mb__estimator__bootstrap': True, 'mb__estimator__max_features': 'log2', 'mb__estimator__oob_score': True} 4.441464313316677
{'mb__estimator__bootstrap': True, 'mb__estimator__max_features': 'sqrt', 'mb__estimator__oob_score': True} 4.561513134245196
{'mb__estimator__bootstrap': True, 'mb__estimator__max_features': None, 'mb__estimator__oob_score': True} 4.654243691248629
{'mb__estimator__bootstrap': True, 'mb__estimator__max_features': 'auto', 'mb__estimator__oob_score': True} 4.73100072793285


In [37]:
grd = {
    'mb__estimator__max_features': ['log2'],
    'mb__estimator__bootstrap': [True],
    'mb__estimator__oob_score': [True],
    'mb__estimator__min_samples_leaf': [i for i in range(2, 10, 2)],
    
}
gsc = GridSearchCV(estimator=ppl, param_grid=grd, scoring=make_scorer(average_lowest_correct), n_jobs=-1, cv=2, verbose=2, error_score=0)
gsc.fit(tfidf, ohe)
prn_rsl(gsc)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__oob_score=True 
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__oob_score=True 
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=4, mb__estimator__oob_score=True 
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=4, mb__estimator__oob_score=True 
[CV]  mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=4, mb__estimator__oob_score=True, total=   4.8s
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=6, mb__estimator__oob_score=True 
[CV]  mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__

[Parallel(n_jobs=-1)]: Done   6 out of   8 | elapsed:   11.2s remaining:    3.7s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:   11.2s finished


{'mb__estimator__bootstrap': True, 'mb__estimator__max_features': 'log2', 'mb__estimator__min_samples_leaf': 2, 'mb__estimator__oob_score': True} 3.5931383342020595
{'mb__estimator__bootstrap': True, 'mb__estimator__max_features': 'log2', 'mb__estimator__min_samples_leaf': 4, 'mb__estimator__oob_score': True} 3.9508392571490196
{'mb__estimator__bootstrap': True, 'mb__estimator__max_features': 'log2', 'mb__estimator__min_samples_leaf': 6, 'mb__estimator__oob_score': True} 4.516813788486079
{'mb__estimator__bootstrap': True, 'mb__estimator__max_features': 'log2', 'mb__estimator__min_samples_leaf': 8, 'mb__estimator__oob_score': True} 4.745740806477704


In [38]:
grd = {
    'mb__estimator__max_features': ['log2'],
    'mb__estimator__bootstrap': [True],
    'mb__estimator__oob_score': [True],
    'mb__estimator__min_samples_leaf': [2],
    'mb__estimator__min_samples_split': [i for i in range(2, 10, 2)],
}
gsc = GridSearchCV(estimator=ppl, param_grid=grd, scoring=make_scorer(average_lowest_correct), n_jobs=-1, cv=2, verbose=2, error_score=0)
gsc.fit(tfidf, ohe)
prn_rsl(gsc)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__min_samples_split=2, mb__estimator__oob_score=True 
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__min_samples_split=2, mb__estimator__oob_score=True 
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__min_samples_split=4, mb__estimator__oob_score=True 
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__min_samples_split=4, mb__estimator__oob_score=True 
[CV]  mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__min_samples_split=2, mb__estimator__oob_score=True, total=   5.1s
[CV]  mb__estimator__bootstrap=True, mb__estimator__max_features=l

[Parallel(n_jobs=-1)]: Done   6 out of   8 | elapsed:   11.7s remaining:    3.9s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:   11.7s finished


{'mb__estimator__bootstrap': True, 'mb__estimator__max_features': 'log2', 'mb__estimator__min_samples_leaf': 2, 'mb__estimator__min_samples_split': 2, 'mb__estimator__oob_score': True} 3.521607698114564
{'mb__estimator__bootstrap': True, 'mb__estimator__max_features': 'log2', 'mb__estimator__min_samples_leaf': 2, 'mb__estimator__min_samples_split': 4, 'mb__estimator__oob_score': True} 3.521607698114564
{'mb__estimator__bootstrap': True, 'mb__estimator__max_features': 'log2', 'mb__estimator__min_samples_leaf': 2, 'mb__estimator__min_samples_split': 6, 'mb__estimator__oob_score': True} 3.5835375963837013
{'mb__estimator__bootstrap': True, 'mb__estimator__max_features': 'log2', 'mb__estimator__min_samples_leaf': 2, 'mb__estimator__min_samples_split': 8, 'mb__estimator__oob_score': True} 3.5904327381059367


In [39]:
grd = {
    'mb__estimator__max_features': ['log2'],
    'mb__estimator__bootstrap': [True],
    'mb__estimator__oob_score': [True],
    'mb__estimator__min_samples_leaf': [2],
    'mb__estimator__min_samples_split': [2],
    'mb__estimator__min_samples_leaf': [i for i in range(1,6)],
    
}
gsc = GridSearchCV(estimator=ppl, param_grid=grd, scoring=make_scorer(average_lowest_correct), n_jobs=-1, cv=2, verbose=2, error_score=0)
gsc.fit(tfidf, ohe)
prn_rsl(gsc)

Fitting 2 folds for each of 5 candidates, totalling 10 fits
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=1, mb__estimator__min_samples_split=2, mb__estimator__oob_score=True 
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=1, mb__estimator__min_samples_split=2, mb__estimator__oob_score=True 
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__min_samples_split=2, mb__estimator__oob_score=True 
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__min_samples_split=2, mb__estimator__oob_score=True 
[CV]  mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__min_samples_split=2, mb__estimator__oob_score=True, total=   5.6s
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=l

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   16.1s finished


{'mb__estimator__bootstrap': True, 'mb__estimator__max_features': 'log2', 'mb__estimator__min_samples_leaf': 2, 'mb__estimator__min_samples_split': 2, 'mb__estimator__oob_score': True} 3.604197184427629
{'mb__estimator__bootstrap': True, 'mb__estimator__max_features': 'log2', 'mb__estimator__min_samples_leaf': 3, 'mb__estimator__min_samples_split': 2, 'mb__estimator__oob_score': True} 3.748951349821162
{'mb__estimator__bootstrap': True, 'mb__estimator__max_features': 'log2', 'mb__estimator__min_samples_leaf': 4, 'mb__estimator__min_samples_split': 2, 'mb__estimator__oob_score': True} 4.081183609827992
{'mb__estimator__bootstrap': True, 'mb__estimator__max_features': 'log2', 'mb__estimator__min_samples_leaf': 5, 'mb__estimator__min_samples_split': 2, 'mb__estimator__oob_score': True} 4.221643196074554
{'mb__estimator__bootstrap': True, 'mb__estimator__max_features': 'log2', 'mb__estimator__min_samples_leaf': 1, 'mb__estimator__min_samples_split': 2, 'mb__estimator__oob_score': True} 4.7

In [41]:
grd = {
    'mb__estimator__max_features': ['log2'],
    'mb__estimator__bootstrap': [True],
    'mb__estimator__oob_score': [True],
    'mb__estimator__min_samples_leaf': [2],
    'mb__estimator__min_samples_split': [2],
    'mb__estimator__min_samples_leaf': [2],
    'mb__estimator__min_weight_fraction_leaf': numpy.linspace(0, 1, 6)
}
gsc = GridSearchCV(estimator=ppl, param_grid=grd, scoring=make_scorer(average_lowest_correct), n_jobs=-1, cv=2, verbose=2, error_score=0)
gsc.fit(tfidf, ohe)
prn_rsl(gsc)

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__min_samples_split=2, mb__estimator__min_weight_fraction_leaf=0.0, mb__estimator__oob_score=True 
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__min_samples_split=2, mb__estimator__min_weight_fraction_leaf=0.0, mb__estimator__oob_score=True 
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__min_samples_split=2, mb__estimator__min_weight_fraction_leaf=0.2, mb__estimator__oob_score=True 
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__min_samples_split=2, mb__estimator__min_weight_fraction_leaf=0.2, mb__estimator__oob_score=True 
[CV]  mb__estimator__bootstrap=True, mb__estimator__max_features=log2, m

[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    9.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    9.7s finished


{'mb__estimator__bootstrap': True, 'mb__estimator__max_features': 'log2', 'mb__estimator__min_samples_leaf': 2, 'mb__estimator__min_samples_split': 2, 'mb__estimator__min_weight_fraction_leaf': 0.6000000000000001, 'mb__estimator__oob_score': True} 0.0
{'mb__estimator__bootstrap': True, 'mb__estimator__max_features': 'log2', 'mb__estimator__min_samples_leaf': 2, 'mb__estimator__min_samples_split': 2, 'mb__estimator__min_weight_fraction_leaf': 0.8, 'mb__estimator__oob_score': True} 0.0
{'mb__estimator__bootstrap': True, 'mb__estimator__max_features': 'log2', 'mb__estimator__min_samples_leaf': 2, 'mb__estimator__min_samples_split': 2, 'mb__estimator__min_weight_fraction_leaf': 1.0, 'mb__estimator__oob_score': True} 0.0
{'mb__estimator__bootstrap': True, 'mb__estimator__max_features': 'log2', 'mb__estimator__min_samples_leaf': 2, 'mb__estimator__min_samples_split': 2, 'mb__estimator__min_weight_fraction_leaf': 0.0, 'mb__estimator__oob_score': True} 3.6301634928195265
{'mb__estimator__boots

## Process Step

In [42]:
tfidf = pandas.read_csv('07 NM PS TFIDF.csv', delimiter=',', encoding='latin-1').fillna('')
tfidf[:3]

,_____________________,_____________________ follow,able,acceptable,accessory,accessory require,accidentally,accompany,accord,account,...,wrong ct,wrong info,wrong info mri,wrong patient,wrong pt,wrong set,wrong tattoo,xrt,xrt date,xrt pt
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
ohe = pandas.read_csv('07 NM PS OHE.csv', delimiter=',', encoding='latin-1').fillna('')
ohe[:3]

,Treatment delivery,Radiation treatment prescription scheduling,Pre-treatment quality assurance,Post-treatment completion,Interventional procedure for planning and/or delivery,Contouring and planning,Imaging for treatment planning,Patient medical consultation and physician assessment,On-treatment quality assurance
0,1,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0


In [44]:
grd = {
    'mb__estimator__max_features': ['log2'],
    'mb__estimator__bootstrap': [True],
    'mb__estimator__oob_score': [True],
    'mb__estimator__min_samples_leaf': [2],
    'mb__estimator__min_samples_split': [2],
    'mb__estimator__min_samples_leaf': [2],
    'mb__estimator__min_weight_fraction_leaf': [0]
}
gsc = GridSearchCV(estimator=ppl, param_grid=grd, scoring=make_scorer(average_lowest_correct), n_jobs=-1, cv=2, verbose=2, error_score=0)
gsc.fit(tfidf, ohe)
prn_rsl(gsc)

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__min_samples_split=2, mb__estimator__min_weight_fraction_leaf=0, mb__estimator__oob_score=True 
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__min_samples_split=2, mb__estimator__min_weight_fraction_leaf=0, mb__estimator__oob_score=True 
[CV]  mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__min_samples_split=2, mb__estimator__min_weight_fraction_leaf=0, mb__estimator__oob_score=True, total=   2.3s
[CV]  mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__min_samples_split=2, mb__estimator__min_weight_fraction_leaf=0, mb__estimator__oob_score=True, total=   2.4s


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    3.2s finished


{'mb__estimator__bootstrap': True, 'mb__estimator__max_features': 'log2', 'mb__estimator__min_samples_leaf': 2, 'mb__estimator__min_samples_split': 2, 'mb__estimator__min_weight_fraction_leaf': 0, 'mb__estimator__oob_score': True} 1.846518987341772


## Contributing Factors

In [46]:
tfidf = pandas.read_csv('07 NM CF TFIDF.csv', delimiter=',', encoding='latin-1').fillna('')
ohe = pandas.read_csv('07 NM CF OHE.csv', delimiter=',', encoding='latin-1').fillna('')

In [47]:
grd = {
    'mb__estimator__max_features': ['log2'],
    'mb__estimator__bootstrap': [True],
    'mb__estimator__oob_score': [True],
    'mb__estimator__min_samples_leaf': [2],
    'mb__estimator__min_samples_split': [2],
    'mb__estimator__min_samples_leaf': [2],
    'mb__estimator__min_weight_fraction_leaf': [0]
}
gsc = GridSearchCV(estimator=ppl, param_grid=grd, scoring=make_scorer(average_lowest_correct), n_jobs=-1, cv=2, verbose=2, error_score=0)
gsc.fit(tfidf, ohe)
prn_rsl(gsc)

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__min_samples_split=2, mb__estimator__min_weight_fraction_leaf=0, mb__estimator__oob_score=True 
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__min_samples_split=2, mb__estimator__min_weight_fraction_leaf=0, mb__estimator__oob_score=True 
[CV]  mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__min_samples_split=2, mb__estimator__min_weight_fraction_leaf=0, mb__estimator__oob_score=True, total=   6.0s
[CV]  mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__min_samples_split=2, mb__estimator__min_weight_fraction_leaf=0, mb__estimator__oob_score=True, total=   6.2s


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    7.3s finished


{'mb__estimator__bootstrap': True, 'mb__estimator__max_features': 'log2', 'mb__estimator__min_samples_leaf': 2, 'mb__estimator__min_samples_split': 2, 'mb__estimator__min_weight_fraction_leaf': 0, 'mb__estimator__oob_score': True} 5.853003283670113


## Overall Severity

In [49]:
tfidf = pandas.read_csv('07 NM OS TFIDF.csv', delimiter=',', encoding='latin-1').fillna('')
ohe = pandas.read_csv('07 NM OS OHE.csv', delimiter=',', encoding='latin-1').fillna('')

In [50]:
grd = {
    'mb__estimator__max_features': ['log2'],
    'mb__estimator__bootstrap': [True],
    'mb__estimator__oob_score': [True],
    'mb__estimator__min_samples_leaf': [2],
    'mb__estimator__min_samples_split': [2],
    'mb__estimator__min_samples_leaf': [2],
    'mb__estimator__min_weight_fraction_leaf': [0]
}
gsc = GridSearchCV(estimator=ppl, param_grid=grd, scoring=make_scorer(average_lowest_correct), n_jobs=-1, cv=2, verbose=2, error_score=0)
gsc.fit(tfidf, ohe)
prn_rsl(gsc)

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__min_samples_split=2, mb__estimator__min_weight_fraction_leaf=0, mb__estimator__oob_score=True 
[CV] mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__min_samples_split=2, mb__estimator__min_weight_fraction_leaf=0, mb__estimator__oob_score=True 
[CV]  mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__min_samples_split=2, mb__estimator__min_weight_fraction_leaf=0, mb__estimator__oob_score=True, total=   1.4s
[CV]  mb__estimator__bootstrap=True, mb__estimator__max_features=log2, mb__estimator__min_samples_leaf=2, mb__estimator__min_samples_split=2, mb__estimator__min_weight_fraction_leaf=0, mb__estimator__oob_score=True, total=   1.4s


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    2.4s finished


{'mb__estimator__bootstrap': True, 'mb__estimator__max_features': 'log2', 'mb__estimator__min_samples_leaf': 2, 'mb__estimator__min_samples_split': 2, 'mb__estimator__min_weight_fraction_leaf': 0, 'mb__estimator__oob_score': True} 1.1131199364257476


## Results
PS 1.84

PT 3.63 from 4.86

CF 5.85

OS 1.11